In [591]:
import stl.mesh
import numpy
import trimesh
import random
import itertools

import meshutil
import meshgen
import examples
import cage

import importlib
importlib.reload(examples)

<module 'examples' from '/home/hodapp/dev/automata_scratch/examples.py'>

In [592]:
importlib.reload(cage)
c = cage.Cage.from_arrays([[0,0,0], [1,1,1]], [[2,2,2]])
c.splits

[0, 2]

In [593]:
c.verts[1:-1,:]

array([[1., 1., 1.]])

In [594]:
for i in c.polys():
    print(i)

[[0. 0. 0.]
 [1. 1. 1.]]
[[2. 2. 2.]]


In [595]:
center = meshutil.Transform().translate(-0.5, -0.5, 0)
cage0 = cage.Cage.from_arrays([
    [0, 0, 0],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 0],
])#.transform(center)

In [596]:
cs,_,_ = cage0.subdivide_deprecated()
c1,c2,c3,c4 = cs

In [597]:
c1.verts

array([[0. , 0. , 0. ],
       [0.5, 0. , 0. ],
       [0.5, 0.5, 0. ],
       [0. , 0.5, 0. ]])

In [598]:
c2.verts

array([[0.5, 0. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0.5, 0. ],
       [0.5, 0.5, 0. ]])

In [599]:
c3.verts

array([[0.5, 0.5, 0. ],
       [1. , 0.5, 0. ],
       [1. , 1. , 0. ],
       [0.5, 1. , 0. ]])

In [600]:
c4.verts

array([[0. , 0.5, 0. ],
       [0.5, 0.5, 0. ],
       [0.5, 1. , 0. ],
       [0. , 1. , 0. ]])

In [601]:
overlap = cage0.classify_overlap(cs)
overlap

[array([3, 1, 2, 1], dtype=uint8),
 array([1, 3, 1, 2], dtype=uint8),
 array([2, 1, 3, 1], dtype=uint8),
 array([1, 2, 1, 3], dtype=uint8)]

In [602]:
# Pretty-print a little:
for i,(c,ols) in enumerate(zip(cs, overlap)):
    print("Cage {}:".format(i))
    for j,(vert,ol) in enumerate(zip(c.verts, ols)):
        print("    Vertex {}: type={} for position {}".format(j, ol, vert))

Cage 0:
    Vertex 0: type=3 for position [0. 0. 0.]
    Vertex 1: type=1 for position [0.5 0.  0. ]
    Vertex 2: type=2 for position [0.5 0.5 0. ]
    Vertex 3: type=1 for position [0.  0.5 0. ]
Cage 1:
    Vertex 0: type=1 for position [0.5 0.  0. ]
    Vertex 1: type=3 for position [1. 0. 0.]
    Vertex 2: type=1 for position [1.  0.5 0. ]
    Vertex 3: type=2 for position [0.5 0.5 0. ]
Cage 2:
    Vertex 0: type=2 for position [0.5 0.5 0. ]
    Vertex 1: type=1 for position [1.  0.5 0. ]
    Vertex 2: type=3 for position [1. 1. 0.]
    Vertex 3: type=1 for position [0.5 1.  0. ]
Cage 3:
    Vertex 0: type=1 for position [0.  0.5 0. ]
    Vertex 1: type=2 for position [0.5 0.5 0. ]
    Vertex 2: type=1 for position [0.5 1.  0. ]
    Vertex 3: type=3 for position [0. 1. 0.]


In [603]:
cf = cage.CageFork(None,
                   numpy.array([[0, 0, 0], [0.5, 0, 0], [1, 0, 0], [1, 0.5, 0],
                                [1, 1, 0], [0.5, 1, 0], [0, 1, 0], [0, 0.5, 0]]) + [0, 0, 2],
                   [[7, 0, 1], [1, 2, 3], [3, 4, 5], [5, 6, 7]]
                  )
mesh2 = cf.transition_from(cage0)

In [604]:
import cProfile, pstats

In [605]:
#mesh = examples.ram_horn()
#mesh = examples.twist()
#mesh = examples.twist_nonlinear()
#mesh = examples.twist_from_gen()
#mesh = examples.twisty_torus(1000)
#mesh = examples.ram_horn2()
#mesh = examples.branch_test()
mesh = examples.ram_horn_branch()

In [606]:
def gen_wave(freq=2.0, a=2.0, d_ph=0.05, count=80):
    ph = 0.0
    while True:
        xs = numpy.linspace(-10, 10, count)
        ys = numpy.zeros(count)
        zs = (numpy.sin(xs*freq + a*numpy.sin(ph)*numpy.pi) + 2)/2
        b1 = numpy.stack((xs,ys,zs), axis=-1)
        b2 = numpy.stack((xs,ys,zs - 2), axis=-1)
        yield [numpy.vstack((b1, b2[::-1,:]))]
        ph += d_ph

#gen = meshgen.gen_inc_y(gen_wave())
#mesh = meshgen.gen2mesh(gen, 200, False, False, True, True)
# The closing method in gen2mesh is inappropriate because
# my boundary is not convex!

In [607]:
def spiral3():
    b = numpy.array([
        [0, 0, 0],
        [1, 0, 0],
        [1, 0, 1],
        [0, 0, 1],
    ], dtype=numpy.float64) - [0.5, 0, 0.5]
    b *= 0.3
    b = meshutil.subdivide_boundary(b)
    b = meshutil.subdivide_boundary(b)
    bs = [b]
    # since it needs a generator:
    gen1 = itertools.repeat(bs)
    gen2 = meshgen.gen_twisted_boundary(gen1, ang=-0.2, dx0=0.5)
    gen3 = meshgen.gen_twisted_boundary(gen2, ang=0.07, dx0=1)
    gen4 = meshgen.gen_twisted_boundary(gen3, ang=-0.03, dx0=3)
    gen = meshgen.gen_inc_y(gen4, dy=0.1)
    mesh = meshgen.gen2mesh(
        gen, count=500, flip_order=True, close_first=True, close_last=True)
    return mesh

def profile():
    pr = cProfile.Profile()
    pr.enable()
    spiral3()
    pr.disable()
    ps = pstats.Stats(pr)
    return ps

#ps = profile()
#ps.sort_stats('tottime').print_stats()

In [608]:
# Work around some annoying-ass trimesh/threejs bug:
wtf = meshutil.Transform().scale(0.1).translate(15,0,0)
center = meshutil.Transform().translate(-0.5, -0.5, -0.5)
base = meshutil.cube(open_xz=False).transform(center)
base = base.transform(wtf)
mesh_out = mesh
# to enable:
#mesh_out = mesh_out.concat(base)

mesh_fname = "branch.stl"
mesh_out.to_stl_mesh().save(mesh_fname)

In [609]:
m = trimesh.Trimesh(vertices=mesh_out.v, faces=mesh_out.f)
# Also acceptable:
#m = trimesh.load_mesh(mesh_fname)

#m.show()
scene = trimesh.Scene([m])
scene.show()